- Show sums of inputs/outputs for only one Anlage
- Show how many entries do have percentage but no total numbers for all inputs/outputs

In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.sankey as sankey
import plotly.graph_objects as go

In [153]:
path1 = 'database1.csv'
path2 = 'database2.csv'
path3 = 'database3.csv'

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)

# Replace NaN in cells with emptry string in all dataframes
# df1.fillna('', inplace=True)
# df2.fillna('', inplace=True)
# df3.fillna('', inplace=True)

In [154]:
# To show all columns
pd.set_option('display.max_columns', None)
df1 = df1.fillna('')
df1["ANLAGE"]=df1["BETRIEB_NAME"] + " " + df1["BETRIEB_ORT"]


print("DataFrame from database1.csv:\n")
print(df1.head(10))

DataFrame from database1.csv:

                                 BETRIEB_NAME      BETRIEB_STRASSE  \
0                            technocycle GmbH     Industriestrasse   
1                    Lars Walch GmbH & Co. KG     Raiffeisenstraße   
2         Lars Walch GmbH & Co. GmbH & Co. KG    Raiffeisenstraße    
3                    Lars Walch GmbH & Co. KG     Raiffeisenstraße   
4                    Lars Walch GmbH & Co. KG    Raiffeisenstraße    
5          FSD Lwerk Berlin Brandenburg gGmbH       Kamenzer Damm    
6  HKV Elektroschrott Recycling GmbH & Co. KG        Gutenbergstr.   
7                     Hausgeräte Lagerverkauf         Wenger Höfe    
8            BRAL Reststoff-Bearbeitungs GmbH    Marzahner Straße    
9          Weinand Recyclingges. mbH & Co. KG  Schwabenheimer Weg    

  BETRIEB_HAUSNR BETRIEB_PLZ    BETRIEB_ORT   BETRIEB_BUNDESLAND BETRIEB_LAND  \
0             60       49565       Bramsche        Niedersachsen                
1             24       91460     Bau

C:\Users\Felix Medicus\AppData\Local\Temp\ipykernel_17872\2160688165.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



## Bilanzen für Gerätekategorie x

In [155]:
def sum_float_values(column):
    total = 0.0
    for value in column:
        if isinstance(value, (float, int)):
            total += value
    return total

In [156]:
# Funktion um zu schauen ob Totaler Eintrag oder Prozent Eintrag leer ist

def determine_missing_vals(dataframe, total_value_col, perc_value_col):

    missing_values = {"total_missing": 0, "perc_missing": 0}
    
    # Check if the specified columns exist in the DataFrame
    if total_value_col not in dataframe.columns or perc_value_col not in dataframe.columns:
        raise ValueError("One or both of the specified columns do not exist in the DataFrame.")

    # Iterate through the rows and perform the comparison
    for index, row in dataframe.iterrows():
        if row[total_value_col] == '' and row[perc_value_col] != '':
            missing_values["total_missing"] += 1
        elif row[total_value_col] != '' and row[perc_value_col] == '':
            missing_values["perc_missing"] += 1

    return missing_values

### Nach einzelnen Anlagen Filtern

In [157]:
# Nach einzelnen Anlagen (Name Betrieb + Ort des Betriebs) schauen. Falls alle Anlagen betrachtet werden sollen das Feld leer lassen.
anlage = "BRAL Reststoff-Bearbeitungs GmbH Berlin"

if len(anlage) != 0:
    df1 = df1[df1['ANLAGE'] == anlage]

### Gesamter Input für Gerätekategorie x in EBA

In [158]:
category = "KG1"
inputCategory = category + "_INPUT"
# Gesamter Input 
KG_Input_Sum = sum_float_values(df1[inputCategory])
print(KG_Input_Sum)

863.0


### Output der EBA für Gerätekategorie x

In [159]:
column = category + "_OUT_GES_MASSE"
ges_out_eba = sum_float_values(df1[column])
print("Gesamter Output der EBA für Kategorie", category, ": ", ges_out_eba)

column = category + "_OUT_KS_MASSE"
ks_out_eba = sum_float_values(df1[column])
print("Davon Kunststoffe", category, ": ", ks_out_eba)

Gesamter Output der EBA für Kategorie KG1 :  863.0
Davon Kunststoffe KG1 :  224.0


## EBA die Output in Prozent aber nicht Tonnen haben

In [160]:
df_filtered = df1[df1[inputCategory] != '']

determine_missing_vals(df_filtered, "KG1_OUT_KS_MASSE", "KG1_OUT_KS_MASSE_KONTR")

{'total_missing': 0, 'perc_missing': 0}

### Input in VWA für Gerätekategorie x

In [161]:
column = category + "_TRT_GES"
ges_in_vwa = sum_float_values(df1[column])
print("Gesamter Input für VWA für Kategorie", category, ": ", ges_in_vwa)

column = category + "_TRT_KS"
ks_in_vwa = sum_float_values(df1[column])
print("Kunststoff Input für VWA für Kategorie", category, ": ", ks_in_vwa)

column = category + "_TRT_RCY"
rec_in_vwa = sum_float_values(df1[column])
print("Recycling Input in VWA für Kategorie", category, ": ", rec_in_vwa)

column = category + "_TRT_RCY_KS"
rec_ks_in_vwa = sum_float_values(df1[column])
print("Recycling Kunststoff Input in VWA für Kategorie", category, ": ", rec_ks_in_vwa)

column = category + "_TRT_RCV_OTH"
sonVW_in_vwa = sum_float_values(df1[column])
print("Sonstige Verwertung Input in VWA für Kategorie", category, ": ", sonVW_in_vwa)

column = category + "_TRT_RCV_OTH_KS"
sonVW_ks_in_vwa = sum_float_values(df1[column])
print("Sonstige Verwertung Kunststoff Input in VWA für Kategorie", category, ": ", sonVW_ks_in_vwa)



Gesamter Input für VWA für Kategorie KG1 :  863.0
Kunststoff Input für VWA für Kategorie KG1 :  224.0
Recycling Input in VWA für Kategorie KG1 :  735.0
Recycling Kunststoff Input in VWA für Kategorie KG1 :  125.0
Sonstige Verwertung Input in VWA für Kategorie KG1 :  11.0
Sonstige Verwertung Kunststoff Input in VWA für Kategorie KG1 :  1.0


### Anzahl an Inputwerten in VWA wo entweder Totaler Wert oder prozentualer Wert nicht angegeben ist

In [162]:
df_filtered = df1[df1[inputCategory] != '']

determine_missing_vals(df_filtered, "KG1_TRT_KS", "")

{'total_missing': 0, 'perc_missing': 0}

### Outputs der VWA für Gerätekategorie x

In [163]:
column = category + "_OUT_RCY_GES"
ges_rec_out_vwa = sum_float_values(df1[column])
print("Gesamter Recycling Output der VWA für Kategorie", category, ": ", ges_rec_out_vwa)
column = category + "_OUT_RCY_KS"
ks_rec_out_vwa = sum_float_values(df1[column])
print("Recycling Kunststoff Output der VWA für Kategorie", category, ": ", ks_rec_out_vwa)

Gesamter Recycling Output der VWA für Kategorie KG1 :  735.0
Recycling Kunststoff Output der VWA für Kategorie KG1 :  81.0


### Anzahl an Outputwerten aus VWA wo entweder Totaler Wert oder prozentualer Wert nicht angegeben ist

### Losses

In [164]:
# Gesamter Output der EBA die nicht in VWA eingeht
out_eba_in_vwa = ges_out_eba - ges_in_vwa
print("Output EBA der nicht in Input VWA ist: ", out_eba_in_vwa)

# Kunstoff Output der EBA die nicht in VWA eingeht
ks_out_eba_in_vwa = ks_out_eba - ks_in_vwa
print("Kunstoff output EBA der nicht in Input VWA ist: ", ks_out_eba_in_vwa)

Output EBA der nicht in Input VWA ist:  0.0
Kunstoff output EBA der nicht in Input VWA ist:  0.0


In [165]:
# Anteil des Recycling Input am gesamten Input in VWA
an_ges_rec_in_vwa = rec_in_vwa/ges_in_vwa
print("Anteil des Recycling Inputs am Gesamtinput in die VWA: ", an_ges_rec_in_vwa)

# Anteil des Recycling Inputs am gesamten Input in VWA für Kunststoffe
an_ks_rec_in_vwa = rec_ks_in_vwa/ks_in_vwa
print("Anteil des Recycling Inputs am Gesamtinput in die VWA für Kunststoffe: ", an_ks_rec_in_vwa)

Anteil des Recycling Inputs am Gesamtinput in die VWA:  0.8516801853997682
Anteil des Recycling Inputs am Gesamtinput in die VWA für Kunststoffe:  0.5580357142857143


In [166]:
# Gesamter Recycling Input in VWA vs. gesamter Recycling Output aus VWA
ges_rec_in_vwa_rec_out_vwa = rec_in_vwa - ges_rec_out_vwa
print("Recycling Input in VWA der nicht als recycelter Output auftritt: ", ges_rec_in_vwa_rec_out_vwa)

# Gesamter Recyling Input in VWA vs. gesamter Recycling Output aus VWA für Kunststoffe
ks_rec_in_vwa_rec_out_vwa = rec_ks_in_vwa - ks_rec_out_vwa
print("Recycling Input in VWA der nicht als recycelter Output auftritt für Kunststoffe: ", ks_rec_in_vwa_rec_out_vwa)

Recycling Input in VWA der nicht als recycelter Output auftritt:  0.0
Recycling Input in VWA der nicht als recycelter Output auftritt für Kunststoffe:  44.0


## Prüfen ob totaler Input- Output-Mengen immer angegeben werden falls es einen Input für die jeweilige Anlage gibt

In [167]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = ["A1", "A2", "B1", "B2", "C1", "C2"],
      color = "blue"
    ),
    link = dict(
      source = [0, 1, 0, 2, 3, 3], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [2, 3, 3, 4, 4, 5],
      value = [8, 4, 2, 8, 4, 2]
  ))])

fig.update_layout(title_text="Ströme zwischen Anlagen", font_size=10)
fig.show()